In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 74.0 MB/s eta 0:00:00


In [ ]:
!pip install --q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.4 MB/s eta 0:00:00


In [ ]:
!pip install --q huggingface_hub

In [ ]:
import numpy as np
import pandas as pd
import random
from datasets import Dataset
from sklearn.model_selection import train_test_split
import datasets
from transformers import T5ForConditionalGeneration, AdamW, Trainer, set_seed, TrainingArguments, T5TokenizerFast
from huggingface_hub import notebook_login
from datasets import load_dataset
import torch

In [ ]:
size = 100_000

In [ ]:
def discriminant(a, b, c):
  return (b ** 2) - (4 * a * c)

$(x - a)(x - b) = 0,  x_1 = a,  x_2 = b,  x^2 - x(a + b) + ab = 0 $

$x_1 * x_2 = \dfrac{c}{a}$

$x_1 + x_2 = - \dfrac{b}{a}$

# Equations with one solution:

In [ ]:
a = 5
b = -4
c = 3
f"{a}*x^2 + {b}*x + {c} = 0"

'5*x^2 + -4*x + 3 = 0'

In [ ]:
f"D = {b}^2 - 4 * {a} * {c} = " + str(b**2 - 4 * a * c)

'D = -4^2 - 4 * 5 * 3 = -44'

In [ ]:
f"x = {-b}^2 // (2 * {a}) = " + str(- b // (2 * a))

'x = 4^2 // (2 * 5) = 0'

In [ ]:
b = random.randint(-100, 100)
a = random.randint(100, 100)
while(a == 0):
  a = random.randint(100, 100)
x = 0.0
solutions, coeffs, discr = [], [], []
for i in range(30_000):  
  while(((b ** 2) % (4 * a) != 0) or (- b % (2 * a) != 0)):
    b = random.randint(-100, 100)
    a = random.randint(-100, 100)
    while(a == 0):
      a = random.randint(100, 100)
  c = (b ** 2) // (4 * a)
  x = - b // (2 * a)
  # solutions.append([x])
  coeffs.append(f"{a}*x^2 + {b}*x + {c} = 0")
  discr.append(f"D = {b}^2 - 4 * {a} * {c} = " + str(b**2 - 4 * a * c) + "; " + f"x = {-b}^2 // (2 * {a}) = " + str(- b // (2 * a)))
  a = 1
  b = 1

In [ ]:
# one_sol = pd.DataFrame({'solution' : solutions, 'coefficients' : coeffs, 'discriminant': discr})
one_sol = pd.DataFrame({'text' : coeffs, 'label': discr})
one_sol

,text,label
0,40*x^2 + -80*x + 40 = 0,D = -80^2 - 4 * 40 * 40 = 0; x = 80^2 // (2 * ...
1,-25*x^2 + 100*x + -100 = 0,D = 100^2 - 4 * -25 * -100 = 0; x = -100^2 // ...
2,-33*x^2 + 0*x + 0 = 0,D = 0^2 - 4 * -33 * 0 = 0; x = 0^2 // (2 * -33...
3,94*x^2 + 0*x + 0 = 0,D = 0^2 - 4 * 94 * 0 = 0; x = 0^2 // (2 * 94) = 0
4,-3*x^2 + -96*x + -768 = 0,D = -96^2 - 4 * -3 * -768 = 0; x = 96^2 // (2 ...
...,...,...
29995,-1*x^2 + -78*x + -1521 = 0,D = -78^2 - 4 * -1 * -1521 = 0; x = 78^2 // (2...
29996,2*x^2 + -56*x + 392 = 0,D = -56^2 - 4 * 2 * 392 = 0; x = 56^2 // (2 * ...
29997,1*x^2 + -12*x + 36 = 0,D = -12^2 - 4 * 1 * 36 = 0; x = 12^2 // (2 * 1...
29998,1*x^2 + 100*x + 2500 = 0,D = 100^2 - 4 * 1 * 2500 = 0; x = -100^2 // (2...


# Equations with no solution:

In [ ]:
b = random.randint(-100, 100)
a = random.randint(-100, 100)
while(a == 0):
    a = random.randint(-100, 100)
c = (b ** 2) // (4 * a)
x = np.nan
cnt = 0
solutions, coeffs, discr = [], [], []
while (cnt < 20_000):  
  # print(a, b, c, discriminant(a, b, c))
  if (discriminant(a, b, c) < 0):
    # print(a, b, c, discriminant(a, b, c))
    cnt += 1
    # solutions.append([x])
    coeffs.append(f"{a}*x^2 + {b}*x + {c} = 0")
    discr.append(f"D = {b}^2 - 4 * {a} * {c} = " + str(b**2 - 4 * a * c) + "; " + f"x = " + str(np.nan))
  
  b = random.randint(-100, 100)
  a = random.randint(-100, 100)
  while(a == 0):
    a = random.randint(-100, 100)

  c = (b ** 2) // (4 * a)
  # solutions.append(x)
  # coeffs.append([a, b, c])

In [ ]:
# no_sol = pd.DataFrame({'solution' : solutions, 'coefficients' : coeffs, 'discriminant': discr})
no_sol = pd.DataFrame({'text' : coeffs, 'label': discr})
no_sol

,text,label
0,-63*x^2 + -58*x + -14 = 0,D = -58^2 - 4 * -63 * -14 = -164; x = nan
1,-37*x^2 + -5*x + -1 = 0,D = -5^2 - 4 * -37 * -1 = -123; x = nan
2,-81*x^2 + 19*x + -2 = 0,D = 19^2 - 4 * -81 * -2 = -287; x = nan
3,-68*x^2 + -35*x + -5 = 0,D = -35^2 - 4 * -68 * -5 = -135; x = nan
4,-46*x^2 + -72*x + -29 = 0,D = -72^2 - 4 * -46 * -29 = -152; x = nan
...,...,...
19995,-34*x^2 + 84*x + -52 = 0,D = 84^2 - 4 * -34 * -52 = -16; x = nan
19996,-28*x^2 + -18*x + -3 = 0,D = -18^2 - 4 * -28 * -3 = -12; x = nan
19997,-15*x^2 + 73*x + -89 = 0,D = 73^2 - 4 * -15 * -89 = -11; x = nan
19998,-28*x^2 + 8*x + -1 = 0,D = 8^2 - 4 * -28 * -1 = -48; x = nan


# Equations with two solutions:

In [ ]:
import math

In [ ]:
def is_square(x):
  return x == (x ** 0.5) ** 2

In [ ]:
def is_integer(a, b):
  return a % b == 0

In [ ]:
a = 1
b = 5
c = 6

In [ ]:
f"D = {b}^2 - 4 * {a} * {c} = " + str(b**2 - 4 * a * c) + "; " + f"x1 = ({-b} + ({b**2 - 4 * a * c})**0.5) // (2 * {a}) = " + str((-b + (discriminant(a, b ,c))**0.5) // (2 * a)) + "; " + f"x2 = ({-b} - ({b**2 - 4 * a * c})**0.5) // (2 * {a}) = " + str((-b - (discriminant(a, b ,c))**0.5) // (2 * a)) 

'D = 5^2 - 4 * 1 * 6 = 1; x1 = (-5 + (1)**0.5) // (2 * 1) = -2.0; x2 = (-5 - (1)**0.5) // (2 * 1) = -3.0'

In [ ]:
b = random.randint(-100, 100)
a = random.randint(-100, 100)
while(a == 0):
  a = random.randint(-100, 100)

c = (b ** 2) // (4 * a)
x1 = 0.0
x2 = 0.0
cnt = 0
solutions, coeffs, discr = [], [], []
while (cnt < 50_000):  
  if ((b ** 2) > (4 * a * c) and is_square(discriminant(a, b, c))):
    if (is_integer(-b + discriminant(a, b, c), (2 * a)) and is_integer(-b - discriminant(a, b, c), (2 * a))):
      cnt += 1
      x1 = (-b + discriminant(a, b, c)) // (2 * a)
      x2 = (-b - discriminant(a, b, c)) // (2 * a)
      # print(a, b, c, discriminant(a, b, c), x1, x2)
      # solutions.append([x1, x2])
      coeffs.append(f"{a}*x^2 + {b}*x + {c} = 0")
      discr.append(f"D = {b}^2 - 4 * {a} * {c} = " + str(b**2 - 4 * a * c) + "; " + f"x1 = ({-b} + ({b**2 - 4 * a * c})**0.5) // (2 * {a}) = " + str((-b + (discriminant(a, b ,c))**0.5) // (2 * a)) + "; " + f"x2 = ({-b} - ({b**2 - 4 * a * c})**0.5) // (2 * {a}) = " + str((-b - (discriminant(a, b ,c))**0.5) // (2 * a)) )
  
  b = random.randint(-100, 100)
  a = random.randint(-100, 100)
  while(a == 0):
    a = random.randint(-100, 100)

  c = (b ** 2) // (4 * a)
  # solutions.append(x)
  # coeffs.append([a, b, c])

In [ ]:
# two_sol = pd.DataFrame({'solution' : solutions, 'coefficients' : coeffs, 'discriminant': discr})
two_sol = pd.DataFrame({'text' : coeffs, 'label': discr})
two_sol

,text,label
0,3*x^2 + 27*x + 60 = 0,D = 27^2 - 4 * 3 * 60 = 9; x1 = (-27 + (9)**0....
1,17*x^2 + -51*x + 38 = 0,D = -51^2 - 4 * 17 * 38 = 17; x1 = (51 + (17)*...
2,3*x^2 + -15*x + 18 = 0,D = -15^2 - 4 * 3 * 18 = 9; x1 = (15 + (9)**0....
3,3*x^2 + 33*x + 90 = 0,D = 33^2 - 4 * 3 * 90 = 9; x1 = (-33 + (9)**0....
4,43*x^2 + -43*x + 10 = 0,D = -43^2 - 4 * 43 * 10 = 129; x1 = (43 + (129...
...,...,...
49995,1*x^2 + 95*x + 2256 = 0,D = 95^2 - 4 * 1 * 2256 = 1; x1 = (-95 + (1)**...
49996,21*x^2 + 21*x + 5 = 0,D = 21^2 - 4 * 21 * 5 = 21; x1 = (-21 + (21)**...
49997,3*x^2 + -69*x + 396 = 0,D = -69^2 - 4 * 3 * 396 = 9; x1 = (69 + (9)**0...
49998,1*x^2 + -73*x + 1332 = 0,D = -73^2 - 4 * 1 * 1332 = 1; x1 = (73 + (1)**...


# Uploading to HuggingFace

In [ ]:
all_equations1 = two_sol.append(one_sol, ignore_index=True)
all_equations = all_equations1.append(no_sol, ignore_index=True)
all_equations.head(5)

,text,label
0,3*x^2 + 27*x + 60 = 0,D = 27^2 - 4 * 3 * 60 = 9; x1 = (-27 + (9)**0....
1,17*x^2 + -51*x + 38 = 0,D = -51^2 - 4 * 17 * 38 = 17; x1 = (51 + (17)*...
2,3*x^2 + -15*x + 18 = 0,D = -15^2 - 4 * 3 * 18 = 9; x1 = (15 + (9)**0....
3,3*x^2 + 33*x + 90 = 0,D = 33^2 - 4 * 3 * 90 = 9; x1 = (-33 + (9)**0....
4,43*x^2 + -43*x + 10 = 0,D = -43^2 - 4 * 43 * 10 = 129; x1 = (43 + (129...


In [ ]:
df = all_equations.sample(frac=1).reset_index(drop=True)

In [ ]:
df

,text,label
0,1*x^2 + 56*x + 784 = 0,D = 56^2 - 4 * 1 * 784 = 0; x = -56^2 // (2 * ...
1,11*x^2 + -33*x + 24 = 0,D = -33^2 - 4 * 11 * 24 = 33; x1 = (33 + (33)*...
2,1*x^2 + 7*x + 12 = 0,D = 7^2 - 4 * 1 * 12 = 1; x1 = (-7 + (1)**0.5)...
3,-47*x^2 + -18*x + -2 = 0,D = -18^2 - 4 * -47 * -2 = -52; x = nan
4,-37*x^2 + 22*x + -4 = 0,D = 22^2 - 4 * -37 * -4 = -108; x = nan
...,...,...
99995,25*x^2 + 75*x + 56 = 0,D = 75^2 - 4 * 25 * 56 = 25; x1 = (-75 + (25)*...
99996,3*x^2 + 39*x + 126 = 0,D = 39^2 - 4 * 3 * 126 = 9; x1 = (-39 + (9)**0...
99997,1*x^2 + -37*x + 342 = 0,D = -37^2 - 4 * 1 * 342 = 1; x1 = (37 + (1)**0...
99998,-44*x^2 + -40*x + -10 = 0,D = -40^2 - 4 * -44 * -10 = -160; x = nan


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2)

In [ ]:
train_csv = df.to_csv("dataset_train_eq.csv")
test_csv = df.to_csv("dataset_test_eq.csv")

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
my_dataset_dict = datasets.DatasetDict({"train" : train_dataset, "test" : test_dataset})

In [ ]:
my_dataset_dict.push_to_hub("snork-maiden/QuadraticEquations_3.0")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

# SimpleT5

In [ ]:
!pip install --q simplet5

     |████████████████████████████████| 1.3 MB 34.3 MB/s 
     |████████████████████████████████| 3.5 MB 27.7 MB/s 
     |████████████████████████████████| 527 kB 55.5 MB/s 
     |████████████████████████████████| 512 kB 66.3 MB/s 
     |████████████████████████████████| 952 kB 48.1 MB/s 
     |████████████████████████████████| 829 kB 75.4 MB/s 
     |████████████████████████████████| 880 kB 61.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [ ]:
# import
from simplet5 import SimpleT5

# instantiate
model = SimpleT5()

# load (supports t5, mt5, byT5 models)
model.from_pretrained("t5","t5-base")

# train
model.train(train_df=train_df, # pandas dataframe with 2 columns: source_text & target_text
            eval_df=eval_df, # pandas dataframe with 2 columns: source_text & target_text
            source_max_token_len = 512, 
            target_max_token_len = 128,
            batch_size = 8,
            max_epochs = 5,
            use_gpu = True,
            outputdir = "outputs",
            early_stopping_patience_epochs = 0,
            precision = 32
            )

# Training

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = T5ForConditionalGeneration.from_pretrained('t5-base')
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
model.lm_head.out_features = 1

In [ ]:
model.config.vocab_size = 100_000

In [ ]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
dataset = load_dataset("snork-maiden/QuadraticEquations_3.0")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/80000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/snork-maiden___parquet/snork-maiden--QuadraticEquations_3.0-4c7cbe7e2ea9562a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import Unigram

tokenizer = Tokenizer(Unigram())

In [ ]:
from tokenizers import normalizers
from tokenizers.normalizers import NFD, StripAccents

normalizer = normalizers.Sequence([NFD(), StripAccents()])
tokenizer.normalizer = normalizer

In [ ]:
from tokenizers.pre_tokenizers import Whitespace

tokenizer.pre_tokenizer = Whitespace()

In [ ]:
from tokenizers.trainers import UnigramTrainer

trainer = UnigramTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [ ]:
files = [f"./dataset_{split}_eq.csv" for split in ["test", "train"]]

tokenizer.train(files, trainer)

In [ ]:
tokenizer.save("./tokenizer-equations-all.json")

In [ ]:
from transformers import PreTrainedTokenizerFast

In [ ]:
# tokenizer = T5TokenizerFast.from_pretrained('t5-base')
# tokenizer = T5TokenizerFast(tokenizer_file="./tokenizer-equations-all.json")
tokenizer = PreTrainedTokenizerFast(tokenizer_file="./tokenizer-equations-all.json")

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
tokenizer.padding = True

In [ ]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 20000
    })
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 80000
    })
})

In [ ]:
max_input_length = 30
max_target_length = 120

def preprocess_function(examples):

    if tokenizer.pad_token is None:
      tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    tokenizer.padding = True

    model_inputs = tokenizer(
        examples["text"],
        max_length=max_input_length,
        truncation=True, padding=True)
    
    with tokenizer.as_target_tokenizer():
        if tokenizer.pad_token is None:
            tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        labels = tokenizer(
            examples["label"], max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_dataset_test = dataset['test'].map(preprocess_function, batched=True)

In [ ]:
tokenized_dataset_test

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 20000
})

In [ ]:
type(tokenized_dataset_test['attention_mask'][0])

list

In [ ]:
tokenized_dataset_train

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 80000
})

In [ ]:
# def tokenize_function(dataset):
    # return tokenizer([str(x) for x in examples['text']], truncation=True)#, padding="max_length")
    # return tokenizer([x for x in dataset['text']])

# tokenized_dataset_train = dataset['train'].map(tokenize_function, batched=True)
# tokenized_dataset_test = dataset['test'].map(tokenize_function, batched=True)

In [ ]:
tokenized_dataset_train = tokenized_dataset_train.remove_columns(["text", "label", "token_type_ids"])

In [ ]:
tokenized_dataset_test = tokenized_dataset_test.remove_columns(["text", "label", "token_type_ids"])

In [ ]:
tokenized_dataset_train

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 80000
})

In [ ]:
train_dataset = tokenized_dataset_train.shuffle(seed=42).select(range(1000))
eval_dataset = tokenized_dataset_test.shuffle(seed=42).select(range(1000))

In [ ]:
train_dataset

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [ ]:
args = TrainingArguments(
    output_dir='./',
    num_train_epochs=5,
    overwrite_output_dir = True,
    evaluation_strategy = 'steps',         
    learning_rate = 3e-4,                 
    # max_steps = 30000,
    logging_steps = 100,                    
    eval_steps = 5000,                      
    save_steps = 10000,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy',
)

In [ ]:
set_seed(42)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
trainer = Trainer(model=model, train_dataset=train_dataset, eval_dataset=eval_dataset, args=args, data_collator=data_collator)

In [ ]:
trainer.train()

ValueError: ignored